# Tensors
Tensors are a specialized data structure that are very similar to arrays and matrices. In PyTorch, we use tensors to encode the inputs and outputs of a model, as well as the model’s parameters.

**Tensors are similar to NumPy’s ndarrays**, except that tensors can run on GPUs or other hardware accelerators(加速器). In fact, tensors and NumPy arrays can often share the same underlying memory, eliminating the need to copy data (see Bridge with NumPy). Tensors are also optimized for automatic differentiation (we’ll see more about that later in the Autograd section). If you’re familiar with ndarrays, you’ll be right at home with the Tensor API. If not, follow along!

# Initializing a Tensor
Tensors can be initialized in various ways. Take a look at the following examples:

### Directly from data

Tensors can be created directly from data. The data type is automatically inferred.

In [2]:
import torch
import numpy as np


In [4]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)
print(data)
print(x_data)

[[1, 2], [3, 4]]
tensor([[1, 2],
        [3, 4]])


### From a NumPy array
Tensors can be created by Numpy arrays (and vice versa(反之亦可) - see **Bridge with NumPy**).


In [6]:
np_array = np.array(data)
x_np = torch.tensor(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])


### From another tensor:
The new tensor retains the properties(shape, datatype) of the argument tensor, unless explicitly(明显地，显示地) overridden

In [7]:
# retain the properties of x_data
x_ones = torch.ones_like(x_data)
print(f"Ones Tensor: \n  {x_ones}")

#override the datatype of x_data
x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"Random Tensor: \n  {x_rand}")


Ones Tensor: 
  tensor([[1, 1],
        [1, 1]])
Random Tensor: 
  tensor([[0.4879, 0.2263],
        [0.9795, 0.9946]])


### With random or constant values:
`shape` is a tuple of dimensions. In the function below, it determines the dimensionality of the output tensor.

In [8]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(rand_tensor)
print(ones_tensor)
print(zeros_tensor)




tensor([[0.4272, 0.5221, 0.3191],
        [0.0647, 0.0563, 0.7182]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


# Attribute of Tensor
Tensor attributes describe their shape, datatype and the device on which they are stored

In [9]:
tensor = torch.rand((3,4,))
print(f"shape：{tensor.shape}")
print(f"datatype: {tensor.dtype}")
print(f"device: {tensor.device}")

shape：torch.Size([3, 4])
datatype: torch.float32
device: cpu


# Operations on Tensors
Over 1200 tensor operations, including arithmetic(算术), linear algebra, matrix manipulation(transposing, indexing, slicing), sampling and more are comprehensively described [here]((https://pytorch.org/docs/stable/torch.html))

Each of this operations are can be run on the CPU and Accelerator such as CUDA, MPS, MTIA or XPU, if you're using the colab, allocate the accelerator by going to Runtime > Change runtime type > GPU.

By default, tensors are created on the CPU. We need to explicitly move tensors to the accelerator using `.to` method(after checking the accelerator availability).

#### Keep in the mind that copying the large tensors across devices can be expensive in a terms of time and memory!

In [11]:
# We move our tensor to the current accelerator if available
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

print(tensor.device)

mps:0


Try out some of the operations from the list. If you're familiar with numpy api, you'll find the tensor api a breeze(轻而易举的事) to use.

### Standard numpy-like indexing and slicing:

In [19]:
tensor = torch.ones((4,4,))
print(tensor)
print(f'the first row: {tensor[0]}')
print(f'the first column: {tensor[:,0]}')
print(f"the last column: {tensor[..., -1]}")
tensor[:, 1] = 0
print(tensor)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
the first row: tensor([1., 1., 1., 1.])
the first column: tensor([1., 1., 1., 1.])
the last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


Joining tensors You can use torch.cat to concatenate a sequence of tensors along a given dimension. See also [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html>), another tensor joining operator that is subtly（巧妙地，敏锐的） different(稍有不同) from torch.cat.

In [20]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


#### Arithmetic operations


In [21]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` return the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)

# this computes the element-wise product. z1, z2, z3 will have same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)


tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

Single-element tensors If you have a one-element tensor, for example by aggregating(聚合) all values of a tensor into one value, you can convert it to a Python numerical value using `item()`:

In [22]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg))

12.0 <class 'torch.Tensor'>


**In-place operations Operations** that store the result into the operand are called in-place. They are denoted by a `_` suffix. For example: `x.copy_(y)`, `x.t_()`, will change x.

In [23]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


> In-place operations save some memory, but can be problematic when computing derivatives because of an immediate loss of history. Hence, their use is discouraged.
> 就地操作可节省部分内存，但在计算导数时可能因历史数据立即丢失而引发问题。因此，不建议使用此类操作。

# Bridge with NumPy
Tensors on the CPU and NumPy arrays can share their underlying memory locations, and changing one will change the other.

### Tensor to NumPy array

In [25]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n：{n}")

# A change in the tensor reflects in the NumPy array.
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n：[1. 1. 1. 1. 1.]
t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### NumPy array to Tensor

In [27]:
n = np.ones(5)
t = torch.from_numpy(n)
print(f"n: {n}")
print(f"t: {t}")
# Changes in the NumPy array reflects in the tensor.
np.add(n, 1, out=n)
print(f"n: {n}")
print(f"t: {t}")

n: [1. 1. 1. 1. 1.]
t: tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
